<a href="https://colab.research.google.com/github/rfdavid/neural-network-playground/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>AlexNet Architecture</h1>

This is the implementation of AlexNet on PyTorch using CIFAR-10 data.
<a href="https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf" target="_blank">https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf</a>


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
from collections import OrderedDict
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time


class Model(nn.Module):
    # This structure was extracted from official PyTorch repository
'''
  (n + 2*p - f)/s + 1

  n x n image (32x32)
  f x f filter (kernel size)
  p padding
  s stride
'''      
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 11, stride = 4, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(64, 192, kernel_size = 5, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(192, 384, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

'''
   The generic Neural Network class for MNIST dataset
   methods:
    - load_data()
    - run()
    - train()
    - test()
'''
class NeuralNetwork():
    def __init__(self, model = None, loss_fn = None, optimizer = None, debug = True):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.debug = debug
        self.train_dataloader = None
        self.test_dataloader = None

    def train(self):
        correct = 0
        size = len(self.train_dataloader.dataset)
        for batch, (X, y) in enumerate(self.train_dataloader):
            pred = self.model(X)
            loss = self.loss_fn(pred, y)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        correct /= size
        if self.debug:
          print(f"Training Accuracy: {(100*correct):>0.1f}%")

        return correct

    def test(self):
        size = len(self.test_dataloader.dataset)
        correct = 0
        with torch.no_grad():
            for  batch, (X, y) in enumerate(self.test_dataloader):
                pred = self.model(X)
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        correct /= size
        if self.debug:
          print(f"Test Accuracy: {(100*correct):>0.1f}%")

        return correct

    def load_data(self, batch_size = 4):
        transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Resize((63,63))])

        training_data = datasets.CIFAR10(
            root='data',
            train=True,
            download=True,
            transform=transform
        )

        test_data = datasets.CIFAR10(
            root='data',
            train=False,
            download=True,
            transform=transform
        )

        self.train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle = True)
        self.test_dataloader = DataLoader(test_data, batch_size = batch_size)

    def run(self, epochs):
        for t in range(epochs):
            print(f"\nEpoch {t+1}")
            print("-------")
            self.train()
            self.test()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
network = NeuralNetwork()
network.model = Model(10).to(device)
network.load_data()
network.optimizer = torch.optim.SGD(network.model.parameters(), lr = 0.001, momentum = 0.9)
network.loss_fn = nn.CrossEntropyLoss()
network.run(epochs = 10)

Files already downloaded and verified
Files already downloaded and verified

Epoch 1
-------
Training Accuracy: 25.9%
Test Accuracy: 45.5%

Epoch 2
-------
Training Accuracy: 52.3%
Test Accuracy: 59.5%

Epoch 3
-------
Training Accuracy: 62.8%
Test Accuracy: 62.7%

Epoch 4
-------
Training Accuracy: 69.4%
Test Accuracy: 68.4%

Epoch 5
-------
